In [80]:
import pandas as pd 
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import matplotlib.pyplot as plt
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException


In [81]:
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\The God of Cringe\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache
C:\Users\The God of Cringe\AppData\Local\Temp\ipykernel_17384\1503906442.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [82]:
page_url = 'https://stormlightarchive.fandom.com/wiki/Category:Characters'
driver.get(page_url)

In [83]:
# Cookies
accept_button = driver.find_element(By.XPATH, '//div[text()="ACCEPT"]').click()
time.sleep(1)

# Find books

In [84]:
def find_element_decorator(func):
    def function_wrapper(*args):
        try:
            elem = func(*args)
            return elem
        except NoSuchElementException:
            return None
    return function_wrapper

@find_element_decorator
def find_elem(elem, by, value):
    out_elem = elem.find_element(by, value)
    return out_elem


@find_element_decorator
def find_click_element(elem, by, value):
    out_elem = elem.find_element(by, value).click()
    return out_elem

def click_elem(elem):
    try:
        driver.execute_script("arguments[0].click();", elem)
    except NoSuchElementException:
        pass

In [85]:
character_list = []
next_button = driver.find_element(By.CLASS_NAME, 'category-page__pagination-next')

print(next_button)
while next_button is not None:
    character_elems = driver.find_elements(By.CLASS_NAME, 'category-page__member-link')
    character_names = [elem.text for elem in character_elems]
    urls = [elem.get_attribute('href') for elem in character_elems]

    for i, url in enumerate(urls):
        print(character_names[i], url)
        if not 'category' in character_names[i]:
            time.sleep(2)
            driver.get(url)
            time.sleep(2)

            aliases_elem = find_elem(driver, By.XPATH, '//th[text()="Aliases"]/following-sibling::td')

            if aliases_elem is not None:
                print(aliases_elem.text)
            
            character_list.append({'character': character_names[i]})
        
    page_url = 'https://stormlightarchive.fandom.com/wiki/Category:Characters'
    driver.get(page_url)

    # Go to the next page [A-Z]
    try:
        next_button = driver.find_element(By.CLASS_NAME, 'category-page__pagination-next')
        click_elem(next_button)
    except NoSuchElementException:
        next_button = None
        pass

 

df = pd.DataFrame(character_list)
df.head(10)

<selenium.webdriver.remote.webelement.WebElement (session="306e1381203dfc1b67adb2fbdd78f842", element="fd56c667-d74d-4143-847b-9f30f9a993ff")>
Category:17th Shard https://stormlightarchive.fandom.com/wiki/Category:17th_Shard
Abrial https://stormlightarchive.fandom.com/wiki/Abrial
Abrobadar https://stormlightarchive.fandom.com/wiki/Abrobadar
Abronai https://stormlightarchive.fandom.com/wiki/Abronai
Adis https://stormlightarchive.fandom.com/wiki/Adis
Adolin Kholin https://stormlightarchive.fandom.com/wiki/Adolin_Kholin
Adrotagia https://stormlightarchive.fandom.com/wiki/Adrotagia
Aesudan Kholin https://stormlightarchive.fandom.com/wiki/Aesudan_Kholin
Aharat https://stormlightarchive.fandom.com/wiki/Aharat
Ahu https://stormlightarchive.fandom.com/wiki/Ahu
Jezrien
Ala https://stormlightarchive.fandom.com/wiki/Ala
Alabet https://stormlightarchive.fandom.com/wiki/Alabet
Aladar https://stormlightarchive.fandom.com/wiki/Aladar
Alakavish https://stormlightarchive.fandom.com/wiki/Alakavish
Alawa

WebDriverException: Message: unknown error: cannot determine loading status
from unknown error: unexpected command response
  (Session info: chrome=103.0.5060.66)
Stacktrace:
Backtrace:
	Ordinal0 [0x00EC6463+2188387]
	Ordinal0 [0x00E5E461+1762401]
	Ordinal0 [0x00D73D78+802168]
	Ordinal0 [0x00D67210+750096]
	Ordinal0 [0x00D6675A+747354]
	Ordinal0 [0x00D65D3F+744767]
	Ordinal0 [0x00D64C28+740392]
	Ordinal0 [0x00D65228+741928]
	Ordinal0 [0x00D6EF2F+782127]
	Ordinal0 [0x00D79FBB+827323]
	Ordinal0 [0x00D7D310+840464]
	Ordinal0 [0x00D654F6+742646]
	Ordinal0 [0x00D79BF3+826355]
	Ordinal0 [0x00DCCF6D+1167213]
	Ordinal0 [0x00DBC5F6+1099254]
	Ordinal0 [0x00D96BE0+945120]
	Ordinal0 [0x00D97AD6+948950]
	GetHandleVerifier [0x011671F2+2712546]
	GetHandleVerifier [0x0115886D+2652765]
	GetHandleVerifier [0x00F5002A+520730]
	GetHandleVerifier [0x00F4EE06+516086]
	Ordinal0 [0x00E6468B+1787531]
	Ordinal0 [0x00E68E88+1805960]
	Ordinal0 [0x00E68F75+1806197]
	Ordinal0 [0x00E71DF1+1842673]
	BaseThreadInitThu

In [ ]:
df.to_csv('stormlight_characters_raw.csv', index=False)

## Preprocess characters

In [ ]:
import re

df = pd.read_csv('stormlight_characters_raw.csv')

df['character'] = df['character'].apply(lambda x: re.sub('[\()].*?[\)]', '', x))
df['character_main_name'] = df['character'].apply(lambda x: x.split(' ', 1)[0])


df = df.drop_duplicates()
df = df.sort_values(by=['character'])
df = df.reset_index(drop=True)


# Fix names manually in excel
df.to_csv('stormlight_characters.csv', index=False)


In [ ]:
driver = webdriver.Chrome(ChromeDriverManager().install())
page_url = 'https://stormlightarchive.fandom.com/wiki/Shallan_Davar'
driver.get(page_url)

accept_button = driver.find_element(By.XPATH, '//div[text()="ACCEPT"]')
click_elem(accept_button)

aliases_elem = find_elem(driver, By.XPATH, '//th[text()="Aliases"]/following-sibling::td')
print(aliases_elem.text)


[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 103.0.5060
[WDM] - Get LATEST chromedriver version for 103.0.5060 google-chrome
[WDM] - Driver [C:\Users\The God of Cringe\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache
C:\Users\The God of Cringe\AppData\Local\Temp\ipykernel_8656\1349161325.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


Veil, Radiant, Swiftspren, Kishi
